In [1]:
import pandas as pd
input_data = pd.read_csv("/home/ec2-user/sylvia/HighPotentialCustomers/ml/input/data/train/sandbox_query_results_008da601-3002-45dd-867e-7c1c6fb01415_last90days.csv")

In [18]:
import pickle

In [2]:
# ================= Logging Setup =================
import logging
import sys
import os
root_logger = logging.getLogger()
stdout_handler = logging.StreamHandler(sys.stdout)
logFormatter = logging.Formatter(fmt=' %(name)s :: %(levelname)-8s :: %(message)s')
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(logFormatter)
root_logger.addHandler(stdout_handler)

# Set log directory from environment or default
log_path = os.environ.get("AMC_AUDIENCES_LOG_DIR", '/home/ec2-user/sylvia/HighPotentialCustomers/ml/output/data/log/')
os.makedirs(log_path, exist_ok=True)
file_handler = logging.FileHandler(os.path.join(log_path, "logfile.log"))
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logFormatter)
root_logger.addHandler(file_handler)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [3]:
input_data.describe()

,dma_code,postal_code,device_type,browser_family,operating_system,search_campaign_cnt,total_impressions_30d,total_clicks_30d,total_impressions_15d,total_clicks_15d,...,total_clicks_3d,total_conversions_30d,total_revenue_30d,total_quantity_30d,total_conversions_15d,total_revenue_15d,total_quantity_15d,total_conversions_3d,total_revenue_3d,total_quantity_3d
count,0.0,0.0,0.0,0.0,0.0,21340.000000,21340.000000,21340.000000,21340.000000,21340.000000,...,21340.000000,21002.000000,21002.000000,21002.000000,21002.000000,21002.000000,21002.000000,21002.000000,21002.000000,21002.000000
mean,NaN,NaN,NaN,NaN,NaN,1.167948,5.171040,1.039925,3.394705,0.681818,...,0.165839,3.747834,15.188974,0.599371,2.115989,8.663064,0.338158,0.538425,2.507990,0.095800
std,NaN,NaN,NaN,NaN,NaN,0.411084,3.512837,1.155755,2.954221,0.952441,...,0.424213,2.893294,53.117776,2.073814,2.008357,38.377372,1.479933,0.845401,18.180404,0.680713
min,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,1.000000,3.000000,0.000000,1.000000,0.000000,...,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,1.000000,4.000000,1.000000,3.000000,0.000000,...,0.000000,3.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,1.000000,7.000000,2.000000,5.000000,1.000000,...,0.000000,5.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,NaN,NaN,NaN,NaN,NaN,4.000000,30.000000,8.000000,23.000000,7.000000,...,5.000000,19.000000,678.860000,28.000000,14.000000,526.470000,28.000000,7.000000,430.470000,14.000000


In [4]:
input_data['high_potential'] = ((input_data['total_conversions_30d'] > 0) | (input_data['total_revenue_30d'] > 0)).astype(int)

In [5]:
features_to_drop = ["user_id","user_id_type","customer_search_term","dma_code","postal_code","device_type","browser_family","operating_system","total_conversions_30d", "total_revenue_30d","total_quantity_30d","total_conversions_15d","total_revenue_15d","total_conversions_3d","total_revenue_3d","total_quantity_3d","total_quantity_15d"]  
X = input_data.drop(columns=[col for col in features_to_drop if col in input_data.columns], errors='ignore')


In [6]:
X["high_potential"].value_counts()

high_potential
1    19116
0     2224
Name: count, dtype: int64

In [7]:
X.head()

,search_campaign_cnt,total_impressions_30d,total_clicks_30d,total_impressions_15d,total_clicks_15d,total_impressions_3d,total_clicks_3d,last_event_dt_30d,last_conversion_dt_30d,high_potential
0,1,2,1,0,1,0,0,2025-01-20T02:32:43.000Z,2025-01-14T21:15:23.000Z,1
1,1,2,2,1,0,0,0,2025-01-21T17:06:06.000Z,2025-01-14T21:15:23.000Z,1
2,1,1,1,1,0,0,0,2025-01-19T17:36:11.000Z,2025-01-06T14:04:56.000Z,1
3,1,3,0,3,0,2,0,2025-01-20T05:15:50.000Z,2025-01-06T14:04:56.000Z,1
4,2,6,2,5,1,1,0,2025-01-21T16:52:28.000Z,2025-01-06T05:27:05.000Z,1


In [8]:
X['last_event_dt_30d'] = pd.to_datetime(X['last_event_dt_30d'], errors='coerce')
X['last_conversion_dt_30d'] = pd.to_datetime(X['last_conversion_dt_30d'], errors='coerce')
# X['time_diff_days'] = (X['last_event_dt_30d'] - X['last_conversion_dt_30d']).dt.days
# 3. 删除原始日期列
X.drop(columns=['last_event_dt_30d', 'last_conversion_dt_30d'], inplace=True)
# 现在 X 中就有一个整型的 time_diff_days 列了

In [9]:
X.head()

,search_campaign_cnt,total_impressions_30d,total_clicks_30d,total_impressions_15d,total_clicks_15d,total_impressions_3d,total_clicks_3d,high_potential
0,1,2,1,0,1,0,0,1
1,1,2,2,1,0,0,0,1
2,1,1,1,1,0,0,0,1
3,1,3,0,3,0,2,0,1
4,2,6,2,5,1,1,0,1


In [10]:
X.describe()


,search_campaign_cnt,total_impressions_30d,total_clicks_30d,total_impressions_15d,total_clicks_15d,total_impressions_3d,total_clicks_3d,high_potential
count,21340.000000,21340.000000,21340.000000,21340.000000,21340.000000,21340.000000,21340.000000,21340.000000
mean,1.167948,5.171040,1.039925,3.394705,0.681818,0.816261,0.165839,0.895783
std,0.411084,3.512837,1.155755,2.954221,0.952441,1.057257,0.424213,0.305550
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,4.000000,1.000000,3.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,7.000000,2.000000,5.000000,1.000000,1.000000,0.000000,1.000000
max,4.000000,30.000000,8.000000,23.000000,7.000000,8.000000,5.000000,1.000000


In [11]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)

In [12]:

def train_test_data_split(df: pd.DataFrame, test_size=0.2, random_state=42):
    """
    拆分训练集与测试集，并在训练集上进行过采样（随机复制少数类）。
    """
    logger.info(f"Splitting data with test_size={test_size}, random_state={random_state}...")
    if 'high_potential' not in df.columns:
        raise ValueError("Column 'high_potential' not found. Please compute it before splitting.")
    
    # 特征中去掉直接用于生成 high_potential 的列
    # features_to_drop = ['high_potential', 'total_conversions_30d', 'total_revenue_30d']
    features_to_drop = ["high_potential"]  
    X = df.drop(columns=[col for col in features_to_drop if col in df.columns], errors='ignore')
    y = df['high_potential']    
    X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=test_size,
            random_state=random_state,
            stratify=y  # 尽量保证拆分后分布一致
        )
    return X_train, X_test, y_train, y_test

In [13]:
X_train, X_test, y_train, y_test = train_test_data_split(X)

 __main__ :: INFO     :: Splitting data with test_size=0.2, random_state=42...


In [14]:

def train_lightgbm_model(X_train, y_train, X_test, y_test, model_dir):
    """
    使用 LightGBM 训练并在测试集上评估，最后保存模型。
    启用 class_weight='balanced' 以减轻数据不平衡。
    """
    logger.info("Training a LightGBM classifier for high potential customers...")
        # 确保标签均为整数类型
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    model = LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)
    
    logger.info("Evaluating on test data...")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    if len(set(y_test)) > 1:
        auc = roc_auc_score(y_test, y_pred)
    else:
        auc = 0.0
    
    logger.info(
        f"Performance Metrics (Test Set):\n"
        f"  Accuracy: {accuracy:.4f}\n"
        f"  Precision: {precision:.4f}\n"
        f"  Recall: {recall:.4f}\n"
        f"  F1 Score: {f1:.4f}\n"
        f"  ROC AUC: {auc:.4f}"
    )
    
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, "high_potential_model.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    logger.info(f"Model saved at: {model_path}")
    return model


In [19]:
train_lightgbm_model(X_train, y_train, X_test , y_test,"/home/ec2-user/sylvia/HighPotentialCustomers/ml/model")

 __main__ :: INFO     :: Training a LightGBM classifier for high potential customers...
[LightGBM] [Info] Number of positive: 15293, number of negative: 1779
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002461 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 17072, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
 __main__ :: INFO     :: Evaluating on test data...
 __main__ :: INFO     :: Performance Metrics (Test Set):
  Accuracy: 0.6338
  Precision: 0.9477
  Recall: 0.6257
  F1 Score: 0.7537
  ROC AUC: 0.6645
 __main__ :: INFO     :: Model saved at: /home/ec2-user/sylvia/HighPotentialCustomers/ml/model/high_potential_model.pkl


LGBMClassifier(class_weight='balanced', random_state=42)

In [51]:
# for col in X.columns:
#     missing_count = X[col].isnull().sum()
#     print(f"特征 '{col}' 缺失值行数： {missing_count}")

特征 'search_campaign_cnt' 缺失值行数： 0
特征 'total_impressions_30d' 缺失值行数： 0
特征 'total_clicks_30d' 缺失值行数： 0
特征 'total_impressions_15d' 缺失值行数： 0
特征 'total_clicks_15d' 缺失值行数： 0
特征 'total_impressions_3d' 缺失值行数： 0
特征 'total_clicks_3d' 缺失值行数： 0
特征 'high_potential' 缺失值行数： 0


In [16]:
df = X